In [1]:
import re
from collections import Counter
import pymorphy2
from tqdm import tqdm
import os
import pandas as pd
import math
import nltk
import numpy as np

In [2]:
marked_path = '/home/nst/mount/data/share/yd/popular_science_texts_store/ner_markup/final_markup/'

In [3]:
def slurp(path):
    with open(path, 'r') as file_object:
        return file_object.read()

In [4]:
def extract_ne(text):
    pattern = re.compile(r'&(.*?)!&')
    nes = re.findall(pattern, text)
    lemmas = []
    for ne in nes:
        divided = ne.split()
        lemma = [morph.parse(word)[0].normal_form for word in divided]
        lemma = ' '.join(lemma)
        lemmas.append(lemma)
    return lemmas

In [5]:
def range_ne(texts_path, top_number):
    nes = []
    texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            texts.append(marked_text)
    for text in tqdm(texts):
        found_ne = extract_ne(text)
        #print(found_ne)
        nes.extend(found_ne)
    ne_dict = Counter(nes)
    ranged_ne = list(sorted(ne_dict, key = lambda x : x[1], reverse=True))
    print(top_number, "most cited scholars are:\n", ranged_ne[:top_number]) 
    return ranged_ne

## Get NE context

In [6]:
class Context:
    def __init__(self, left_context: str, right_context: str,
                 left_word: str, word: str):
        self.left_context = left_context
        self.right_context = right_context
        self.left_word = left_word
        self.word = word
    def __repr__(self):
        return 'Context("%s", "%s", "%s", "%s")' % (
            self.left_context, self.right_context,
            self.left_word, self.word)

def tokenize_text(text: str) -> [str]:
    return re.findall(r'&?[\w.\'-]+!?&?', text, flags=re.UNICODE)

def cleanup_tag(text: str) -> str:
    return text.replace('&', '').replace('!', '')

def get_span(tokens: [str], start: int, end: int) -> str:
    return cleanup_tag(' '.join(tokens[start:end]))

def get_complete_word(index: int, tokens: [str]) -> (str, int, bool):
    current_word = tokens[index]
    if not current_word.startswith('&'):
        return current_word, index+1, False

    word_parts = [current_word]
    index += 1
    if not current_word.endswith('!&'):
        while index < len(tokens):
            current_word = tokens[index]
            word_parts.append(current_word)
            if current_word.endswith('!&'):
                index += 1
                break
            index += 1
        else:
            raise ValueError('No matching closing tag in tokens: "%s"' % tokens)
    
    word = cleanup_tag(' '.join(word_parts))
    return word, index, True

assert ('a', 1, False) == get_complete_word(0, ['a', 'b'])
assert ('b', 2, False) == get_complete_word(1, ['a', 'b'])
assert ('a', 1, True) == get_complete_word(0, ['&a!&', 'b']), get_complete_word(0, ['&a!&', 'b'])
assert ('a b', 2, True) == get_complete_word(0, ['&a', 'b!&'])
assert ('a b c', 3, True) == get_complete_word(0, ['&a', 'b', 'c!&'])

def extract_contexts(text: str, window_size: int) -> [Context]:
    result = []
    
    tokens = tokenize_text(text)
    index = 0

    while index < len(tokens):
        word, new_index, is_tag = get_complete_word(index, tokens)
        if not is_tag:
            index = new_index
            continue

        left_context = get_span(tokens, index-5, index)
        right_context = get_span(tokens, new_index+1, new_index+6)
        left_word = get_span(tokens, index-1, index)
        
        context = Context(left_context, right_context, left_word, word)
        result.append(context)
        
        index = new_index
    
    return result

#print(extract_contexts(text_3, 5))

In [7]:
text_3 = '«Русский со словарем» — очень забавная, изящная, но вполне серьезная книга. Есть книга &Максима Кронгауза!& «Русский язык на грани нервного срыва», у которой уже второе, переработанное, более полное издание. &Кронгауз!&, кстати, тоже выпускник нашего отделения.'

## Make Dataframe and get all NE contexts

### Marked & Preprocessed DataFrame

In [4]:
def make_texts_df(texts_path):
    marked_texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            marked_texts.append(marked_text)
    marked_df = pd.DataFrame(marked_texts, columns=['marked_texts'])
    return marked_df

In [9]:
marked_df = make_texts_df(marked_path)

In [10]:
marked_df.head()

,marked_texts
0,Как интересно сгруппированы атомы... комплимен...
1,Что люди обычно думают о теоретической лингвис...
2,Радиопульсары – одно из наблюдательных проявле...
3,"Мне уже много лет. Когда я заканчивал школу, э..."
4,Совместно с издательством «Альпина нон-фикшн» ...


### Get all contexts from marked texts with window 5

In [11]:
def compile_contexts_dataframe(texts, window_size):
    left = []
    left_words = []
    entities = []
    right = []
    for text in texts:
        contexts = extract_contexts(text, window_size)
        for context in contexts:
            left_context = context.left_context
            left.append(left_context)
            left_word = context.left_word
            left_words.append(left_word)
            entity = context.word
            entities.append(entity)
            right_context = context.right_context
            right.append(right_context)
    dataframe = pd.DataFrame(np.column_stack([left, left_words, entities, right]),
                            columns=['left_context', 'left_word', 
                                     'named_entity', 'right_context'])
    
    return dataframe
    

In [12]:
def sort_dataframe(dataframe, column):
    return dataframe.sort_values(by=[column], axis=0)

In [13]:
contexts_df = compile_contexts_dataframe(marked_df.marked_texts, 5)

In [14]:
contexts_df

,left_context,left_word,named_entity,right_context
0,Из выступления лауреата Нобелевской премии,премии,Ричарда Фейнмана,Калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,году,Гердом Биннигом,Генрихом Рорером сотрудниками швейцарского отд...
2,1981 году Гердом Биннигом и,и,Генрихом Рорером,швейцарского отделения IBM сконструировавшими ...
3,сотрудниками IBM Almaden Research Center,Center,Дональдом Эйглером,Эрхардом Швейцером в 1990 году
4,Research Center Дональдом Эйглером и,и,Эрхардом Швейцером,1990 году заключался в том
5,подобная работа была проделана группой,группой,Джозефа Стросцио,Национального Института Стандартов и Технологий
6,решётки из атомов подложки Когда,Когда,Дональд Эйглер,назад выступал с лекцией в
7,этом больше всех писал академик,академик,Андрей Анатольевич Зализняк,Дело в том что написать
8,лингвистике. Это прежде всего книга,книга,Зализняка,заметок о любительской лингвистике книга
9,заметок о любительской лингвистике книга,книга,Ирины Левонтиной,со словарем очень забавная изящная


In [15]:
sorted_df = sort_dataframe(contexts_df, 'left_word' )

In [16]:
sorted_df = sorted_df.drop_duplicates()
sorted_df = sorted_df.reset_index()

In [17]:
sorted_df.head()

,index,left_context,left_word,named_entity,right_context
0,1850,,,Меньше,осталось до начала воплощения проекта
1,191,,,Виталий Лазаревич Гинзбург,поистине уникальной личностью. Гениальный ученый
2,1751,,,Альберт Эйнштейн,опубликовал труды сделавшие его знаменитыми
3,641,,,Серджио Канаверо,время лекции в рамках проекта
4,1940,метра под поверхностью Марса .,.,Приборы,и FREND разработаны в Институте


In [48]:
sorted_df.to_csv('ne_contexts_sorted.tsv', sep='\t')

In [101]:
for root, dirs, files in os.walk(marked_path):
    for file_name in files:
        input_path = marked_path + file_name
        marked_text = slurp(input_path)
        if 'Кронгауз' in marked_text:
            print(file_name)

126.txt


In [63]:
sorted_contexts = pd.read_csv('/home/nst/mount/data/linguistics_hse/popular-science-research'\
                              '/popular-science-repo/Popular-Science-Texts-Compling-research/'\
                              'ner_markup/ne_contexts_sorted.tsv', sep='\t', index_col=0)

In [64]:
sorted_contexts.head()

,left_context,left_word,named_entity,right_context
1850,NaN,NaN,Меньше,осталось до начала воплощения проекта
191,NaN,NaN,Виталий Лазаревич Гинзбург,поистине уникальной личностью. Гениальный ученый
1751,NaN,NaN,Альберт Эйнштейн,опубликовал труды сделавшие его знаменитыми
641,NaN,NaN,Серджио Канаверо,время лекции в рамках проекта
1940,метра под поверхностью Марса .,.,Приборы,и FREND разработаны в Институте


## Pos-tagging contexts

In [9]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [22]:
from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [60]:
def tag_pymorphy(text):
    
    def find_tag(raw_tag):
        tag = re.findall('[A-Z]+', raw_tag)
        return tag[0]
    
    words = text.split()
    tags = [find_tag(str(morph.parse(w)[0].tag)) for w in words]
    tagged = []
    for word, tag in zip(words, tags):
        tagged.append(word)
        if (tag == 'UNKN' or tag == 'LATN'):
            tagged.append('')
        else:
            tagged.append(tag)
    return ' '.join(tagged)

In [54]:
def tag_contexts_pymorphy(contexts):
    tagged = []
    for text in tqdm(contexts):
        try:
            text = tag_pymorphy(text)
            tagged.append(text)
        except AttributeError:
            tagged.append(None)
    return tagged


def tag_dataset(dataframe):
    left_contexts = dataframe.left_context
    right_contexts = dataframe.right_context
    tagged_left = tag_contexts_pymorphy(left_contexts)
    tagged_right = tag_contexts_pymorphy(right_contexts)
    dataframe['left_context'] = tagged_left
    dataframe['right_context'] = tagged_right
    return dataframe

In [65]:
%%time
tagged_df = tag_dataset(sorted_contexts)

100%|██████████| 1920/1920 [00:06<00:00, 286.24it/s]

CPU times: user 10.2 s, sys: 120 ms, total: 10.4 s
Wall time: 11.9 s


In [66]:
tagged_df

,left_context,left_word,named_entity,right_context
1850,None,NaN,Меньше,осталось VERB до PREP начала NOUN воплощения N...
191,None,NaN,Виталий Лазаревич Гинзбург,поистине ADVB уникальной ADJF личностью. Гени...
1751,None,NaN,Альберт Эйнштейн,опубликовал VERB труды NOUN сделавшие PRTF его...
641,None,NaN,Серджио Канаверо,время NOUN лекции NOUN в PREP рамках NOUN прое...
1940,метра NOUN под PREP поверхностью NOUN Марса NO...,.,Приборы,и CONJ FREND разработаны PRTS в PREP Институт...
1164,из PREP этой ADJF нелепой ADJF затеи NOUN . PNCT,.,Эддингтон,только PRCL высказывался VERB в PREP пользу NO...
1930,разрешения NOUN 4 NUMB 5 NUMB Мп NOUN . PNCT,.,Она,использована PRTS для PREP создания NOUN точны...
782,и CONJ Уайтом NOUN С. Levin . PNCT,.,R. Lippett,R. К. White в PREP 1939 NUMB
944,курсу NOUN должен ADJS оставить INFN порядок N...,.,Лэйнг,термин NOUN карта NOUN курса NOUN это PRCL экз...
39,под PREP руководством NOUN Гарри NOUN Уиттингт...,.,Саймон Конвей Моррис,Бриггс NOUN Дэвид NOUN Брутон NOUN Крис NOUN Х...


In [67]:
tagged_df.to_csv('ne_pos_contexts.tsv', sep='\t')

In [127]:
def tag_from_file(texts_path):
    texts = []
    tagged = []
    
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            try:
                text = slurp(input_path)
                texts.append(text)
            except FileNotFoundError:
                continue
    for text in tqdm(texts):
        tagged_text = tag_text(text)
        tagged.append(tagged_text)
    return tagged

In [130]:
tagged = tag_from_file('/home/nst/mount/data/share/yd/popular_science_texts_store/nplus1.ru/nplus1_materials/')

  0%|          | 0/542 [00:00<?, ?it/s]

TreeTaggerError: Time out for TreeTagger reply, enable debug / see error logs

## Count most frequent collocations in contexts

In [33]:
def preprocess(texts):
    preproc_texts = []
    pattern = re.compile(r'\w+')
    try:
        for text in tqdm(texts):
            text = text.lower()
            words = ' '.join(re.findall(pattern, text))
            preproc_texts.append(words)
    except AttributeError:
        print('Corrupted:', text)
        preproc_texts.append('')
    return preproc_texts

In [37]:
preprocessed_left = preprocess(df.left_context)
preprocessed_right = preprocess(df.right_context)

100%|██████████| 1216/1216 [00:00<00:00, 93964.03it/s]


In [36]:
df = df.dropna(axis=0, how='any')
len(df)

1216

In [38]:
preprocessed_df = pd.DataFrame(preprocessed_left, columns=['left'])
preprocessed_df['right'] = preprocessed_right
preprocessed_df.head(3)

,left,right
0,из выступления лауреата нобелевской премии,в калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,и генрихом рорером сотрудниками швейцарского о...
2,1981 году гердом биннигом и,сотрудниками швейцарского отделения ibm сконст...


In [44]:
def compile_tokens_list(texts):
    tokens = []
    for text in texts:
        text = text.split()
    tokens.extend(text)
    tokens = ' '.join(tokens)
    return tokens

In [45]:
tokens_left = compile_tokens_list(preprocessed_df.left)
tokens_right = compile_tokens_list(preprocessed_df.right)

In [47]:
def aprior_probability(dictionary):
    s = sum(dictionary.values())
    for word  in dictionary:
        dictionary[word] /= s
   # aprior_dictionary = dict(wiki_freq).get(word)/s
    return dict(dictionary)

In [48]:
def bigram_probability(bigrams):
    count_bigrams = len(bigrams)
    joined_bigrams = [' '.join(bigram) for bigram in bigrams]
    bigrams_dict = Counter(joined_bigrams)
    for bigram in bigrams_dict:
        bigrams_dict[bigram] /= count_bigrams
    return dict(bigrams_dict)

In [49]:
def count_pmi(bigrams, bigrams_prob, words_aprior):
    joined_bigrams = []
    pmis = []
    for bigram in bigrams:
        #print(bigram)
        joined_bigram = ' '.join(bigram)
        pmi = math.log(bigrams_prob.get(joined_bigram)/
                      (words_aprior.get(bigram[0])*words_aprior.get(bigram[1])))
        joined_bigrams.append(joined_bigram)
        pmis.append(pmi)
    counted_pmi = list(zip(joined_bigrams, pmis))
    return counted_pmi

In [51]:
def range_pmi(pmis_list, top_k):
    pmis_dict = dict(pmis_list)
    best_pmis = sorted(pmis_dict, key = lambda x:x[1], reverse = True)
    print(best_pmis[:top_k])
    return best_pmis

In [52]:
def extract_collocations(tokens, top_best):
    freq = Counter(tokens.split())
    bigrams = list(nltk.bigrams(tokens.split()))
    aprior = aprior_probability(freq)
    bigrams_prob = bigram_probability(bigrams)
    pmi = count_pmi(bigrams, bigrams_prob, aprior)
    best = range_pmi(pmi, top_best)
    return best

In [56]:
# top best left 
collocations_left = extract_collocations(tokens_left, 10)

['мы рассказывали', 'совсем недавно', 'недавно мы', 'рассказывали о']


In [57]:
# top best right
collocations_right = extract_collocations(tokens_right, 10)

['истории об', 'архивной находке', 'его истории', 'об архивной', 'и его']


In [213]:
rubrics = pd.read_csv('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/Popular-Science-Texts-Compling-research/rubric_lists/rubrics_mapping.tsv', sep='\t')

In [214]:
rubrics.head(10)

,path,source,url,date,title,subtitle,author,tags,genre,mapped_rubrics,final_rubrics
0,chrdk.ru/articles/sci_10_salt_lakes.txt,https://chrdk.ru/,https://chrdk.ru/sci/10_salt_lakes,16.08.2017,Десять самых известных соленых озер,NaN,Егор Задереев,География_Экология,Статьи,Науки о земле|Науки о земле,Науки о земле
1,chrdk.ru/articles/sci_33_fractures.txt,https://chrdk.ru/,https://chrdk.ru/sci/33_fractures,08.08.2017,33 перелома,Скелет мужчины с 33 переломами нашли археологи...,Екатерина Боровикова,Российская наука_Антропология,Статьи,Мусор|История,История
2,chrdk.ru/articles/sci_46_chromosomes.txt,https://chrdk.ru/,https://chrdk.ru/sci/46_chromosomes,04.08.2017,46 — норма?,Считаем хромосомы: сколько человеку для счасть...,Полина Лосева,Генетика_Медицина,Статьи,Физиология человека|Физиология человека,Физиология человека
3,chrdk.ru/articles/sci_750gev.txt,https://chrdk.ru/,https://chrdk.ru/sci/750gev,23.08.2016,Несбывшиеся надежды на новую физику,Какие «страшные сценарии» физики хотели забыть...,Екатерина Боровикова,Физика_Интервью_Закрытия,Статьи,Физика|Мусор|Мусор,Физика
4,chrdk.ru/articles/sci_alien_anatomy.txt,https://chrdk.ru/,https://chrdk.ru/sci/alien_anatomy,22.05.2017,Анатомия каменных пришельцев,Спецпроект «Чердака» и «Красивой науки»,NaN,Геология_Космос,Статьи,Науки о земле|Космос,Космос|Науки о земле
5,chrdk.ru/articles/sci_alma_mater.txt,https://chrdk.ru/,https://chrdk.ru/sci/alma_mater,17.11.2016,Зарубежные университеты: где учились гении?,"Альма-матер Эйнштейна, Хокинга и других ученых",NaN,Образование,Статьи,Мусор,Мусор
6,chrdk.ru/articles/sci_almost_lifelike.txt,https://chrdk.ru/,https://chrdk.ru/sci/almost_lifelike,13.04.2017,Почти как живой,"Биологи обнаружили гигантский вирус, в геноме ...",Иван Шунин,Науки о живом_Молекулярная биология,Статьи,Биология|Биология,Биология
7,chrdk.ru/articles/sci_AlphaGo_vs_LiSedol.txt,https://chrdk.ru/,https://chrdk.ru/sci/AlphaGo_vs_LiSedol,15.03.2016,"Игры, в которые играли люди",Программа AlphaGo обыграла признанного чемпион...,Михаил Петров,Информационные технологии_Футурология_Технолог...,Статьи,Computer Science|Мусор|Технологии|Computer Sci...,Computer Science
8,chrdk.ru/articles/sci_Alzheimers_news.txt,https://chrdk.ru/,https://chrdk.ru/sci/Alzheimers_news,29.06.2016,"Немец, от которого все без ума",Обзор новых исследований болезни Альцгеймера,Алексей Паевский,Молекулярная биология_Медицина_Выбор редакции,Статьи,Биология|Физиология человека|Мусор,Физиология человека|Биология
9,chrdk.ru/articles/sci_animals_vs_climate.txt,https://chrdk.ru/,https://chrdk.ru/sci/animals_vs_climate,02.12.2016,Погоды не сделают,"Какие животные, кроме людей, влияют на мировой...",Михаил Петров,Науки о живом_Животные_Климат_Изменение климата,Статьи,Биология|Биология|Науки о земле|Науки о земле,Науки о земле|Биология
